In [1]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'


In [ ]:
# here we implement windowed candle data for 1 minute
# here we implement windowed candle data for 3 minute
# here we implement windowed candle data for 5 minute

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic candles-1min 
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  candles-1min  --from-beginning

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic candles-1min 
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  candles-1min  --from-beginning

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic candles-3min 
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  candles-3min  --from-beginning

#     kafka-topics  --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic candles-5min 
#     kafka-console-consumer --bootstrap-server localhost:9092 --topic  candles-5min  --from-beginning

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaCandles").getOrCreate()

In [4]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1

kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "stock-ticks")\
  .option("group.id", "stock-ticks-group")\
  .load()

In [5]:
# key and value are binary type, we need to CAST To STRING type
kafkaDf.printSchema()
# timestampType values
# CreateTime:  Timestamp relates to message creation time as set by a Kafka client/producer
# LogAppendTime : Timestamp relates to the time a message was appended to a Kafka log.
# UnknownType

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [6]:
# Kafka value, key are in binary format
# we need to type cast to STRING, 
# we pick only value, timestamp as we need for 10 minute window
ticksDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp")
ticksDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [8]:
echoOnconsole = ticksDf\
                .writeStream\
                .outputMode("update")\
                .format("console")\
                .start() # start the query. spark will subscribe for data